In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

import polars as pl

import stock

In [ ]:
csv_dir = stock.PROJECT_ROOT / "data" / "daily"
csv_list = sorted(csv_dir.glob("*.csv"))[2:]

nikkei_df = stock.kabutan.read_data_csv(csv_dir / "0000.csv", exclude_none=False)
topix_df = stock.kabutan.read_data_csv(csv_dir / "0010.csv", exclude_none=False)

In [ ]:
def calc_relative_strength(df: pl.DataFrame, ref_df: pl.DataFrame, end: pl.Series):
    start = end - timedelta(days=365)
    if start < df["date"][0]:
        return -1.0
    df = df.filter(pl.col("volume").is_not_nan().is_not_null() & (pl.col("volume") > 0))
    df = df.filter((start < pl.col("date")) & (pl.col("date") <= end))
    ref_df = ref_df.filter((start < pl.col("date")) & (pl.col("date") <= end))
    if len(df) != len(ref_df):
        return -1.0
    
    return stock.relative_strength.relative_strength(
        df["close"].to_numpy(), ref_df["close"].to_numpy()
    )

In [ ]:
for csv_path in csv_list:
    df = stock.kabutan.read_data_csv(csv_path, exclude_none=False)
    new_df = df.with_columns(
        pl.col("date").map_elements(lambda date : calc_relative_strength(df, nikkei_df, date), return_dtype=pl.Float64).alias("rs_nikkei"),
        pl.col("date").map_elements(lambda date : calc_relative_strength(df, topix_df, date), return_dtype=pl.Float64).alias("rs_topix")
    )